In [1]:
# from search import *
import random

In [2]:
class Cromosoma(object):
    
    def __init__(self, tareas_trabajadores, trabajadores_tareasEnOrden):
        self.tareas_trab = tareas_trabajadores
        self.trab_tareasOrden = trabajadores_tareasEnOrden
        
    def __str__(self):
        return "Tareas-Trabajador asignado:   "+str(self.tareas_trab)+"\nTrabajadores-Tareas en orden: "+str(self.trab_tareasOrden)
    

In [3]:
NOT_QUALIFIED = -1

class ProblemaGenetico(object):
    def __init__(self, timetable, restricciones):
        self.working_board = timetable
        self.num_tareas = len(timetable[0])
        self.num_trabajadores = len(timetable)
        self.restricciones_tarea = [[] for _ in range(self.num_tareas)]
        for restriccion in restricciones:
            self.restricciones_tarea[restriccion[1]].append(restriccion[0])
            
    def decodifica(self, cromosoma):
        print(cromosoma)
    
    def muta(self, cromosoma, prob):
        # Reemplazo aleatorio: cambiamos el trabajador de una tarea
        if (random.random() < prob):
            tarea_a_cambiar  = random.choice([i for i in range(self.num_tareas)])
            viejo_trabajador = cromosoma.tareas_trab[tarea_a_cambiar]
            nuevo_trabajador = random.choice([i for i in range(self.num_trabajadores) if self.working_board[i][tarea_a_cambiar]!=NOT_QUALIFIED])
            #Cambiamos cromosoma
            cromosoma.tareas_trab[tarea_a_cambiar] = nuevo_trabajador
            cromosoma.trab_tareasOrden[viejo_trabajador].remove(tarea_a_cambiar)
            cromosoma.trab_tareasOrden[nuevo_trabajador].append(tarea_a_cambiar)
    
    
    def cruza(self, cromosoma1, cromosoma2):  
        
        hijo1 = Cromosoma([0]*self.num_tareas, [[] for _ in range(self.num_trabajadores)])
        hijo2 = Cromosoma([0]*self.num_tareas, [[] for _ in range(self.num_trabajadores)])
        
        for trabajador in range(self.num_trabajadores):
            for tarea in cromosoma1.trab_tareasOrden[trabajador]:
                if tarea <= (self.num_tareas//2):
                    hijo1.tareas_trab[tarea]=trabajador
                    hijo1.trab_tareasOrden[trabajador].append(tarea)
                else:
                    hijo2.tareas_trab[tarea]=trabajador
                    hijo2.trab_tareasOrden[trabajador].append(tarea)
                    
        for trabajador in range(self.num_trabajadores):
            for tarea in cromosoma2.trab_tareasOrden[trabajador]:
                if tarea <= (self.num_tareas//2):
                    hijo2.tareas_trab[tarea]=trabajador
                    hijo2.trab_tareasOrden[trabajador].append(tarea)
                else:
                    hijo1.tareas_trab[tarea]=trabajador
                    hijo1.trab_tareasOrden[trabajador].append(tarea)
    
        return [hijo1,hijo2]
        
    def fitness(self, cromosoma):    
        tareas_tiempoFinalizacion = [-1]*self.num_tareas
        trab_tiempoOcupacion = [0]*self.num_trabajadores
        
        tareasCalculadas = 0
        hayCambio = True
        
        while (hayCambio and tareasCalculadas<self.num_tareas):
            hayCambio = False
            
            for i in range(self.num_trabajadores):
                
                tareas_pendientes = [k for k in cromosoma.trab_tareasOrden[i] if tareas_tiempoFinalizacion[k]==-1]
                
                if(not(tareas_pendientes)):
                    continue
                
                for j in tareas_pendientes:
                    ## Si tiene alguna dependencia por ejecutar paramos y vamos al siguiente trabajador
                    if(len([k for k in self.restricciones_tarea[j] if tareas_tiempoFinalizacion[k]==-1])>0):
                        break
                    ## Si no la ejecutamos
                    else:
                        tareasCalculadas += 1
                        hayCambio = True
                        
                        ## Aux1 = máximo tiempo de finalización de las dependencias
                        aux1 = max([tareas_tiempoFinalizacion[k] for k in self.restricciones_tarea[j]], default=0)
                        ## Aux2 = cuándo queda libre el trabajador que hace la tarea
                        aux2 = trab_tiempoOcupacion[i]
                        aux3 = max(aux1,aux2)
                        
                        tareas_tiempoFinalizacion[j] = aux3 + self.working_board[i][j]
                        trab_tiempoOcupacion[i] = aux3 + self.working_board[i][j]
        
        if tareasCalculadas<self.num_tareas:
            #print("Hemos llegado a una estado no válido:")
            #print(cromosoma)
            return 10000000
        else:
            return max(tareas_tiempoFinalizacion)

In [4]:
grid = [[1,1,1,1],[2,2,2,2],[NOT_QUALIFIED,1,NOT_QUALIFIED,1]]
problema = ProblemaGenetico(grid, [])

In [5]:
## Elegimos de entre los trabajadores cualificados para una determinada tarea uno al azar.

def poblacion_inicial(problema_genetico, size):
    nueva_poblacion = []
    for i in range(size):
        genotipo = Cromosoma([0]*problema_genetico.num_tareas, [[] for _ in range(problema_genetico.num_trabajadores)])
        
        ## Si queremos que de verdad sea aleatorio tenemos que hacer shuffle de cada lista de genotipo!
        for j in range(problema_genetico.num_tareas):
            worker = random.choice([i for i in range(problema_genetico.num_trabajadores) if problema_genetico.working_board[i][j]!=NOT_QUALIFIED])
            genotipo.tareas_trab[j] = worker
            genotipo.trab_tareasOrden[worker].append(j)
        
        nueva_poblacion.append(genotipo)
    return nueva_poblacion

In [6]:
poblacion_ini = poblacion_inicial(problema, 2)
print(poblacion_ini[0])
print(poblacion_ini[1])

Tareas-Trabajador asignado:   [0, 0, 1, 1]
Trabajadores-Tareas en orden: [[0, 1], [2, 3], []]
Tareas-Trabajador asignado:   [0, 2, 0, 0]
Trabajadores-Tareas en orden: [[0, 2, 3], [], [1]]


In [7]:
cromosoma = Cromosoma([0,0,1],[[0,1],[2]])

In [8]:
grid = [[10,20,-1],[-1,-1,30]]
problemaAux = ProblemaGenetico(grid, [(1,0)])

In [9]:
print(problema.cruza(poblacion_ini[0],poblacion_ini[1])[0])
print(problema.cruza(poblacion_ini[0],poblacion_ini[1])[1])

Tareas-Trabajador asignado:   [0, 0, 1, 0]
Trabajadores-Tareas en orden: [[0, 1, 3], [2], []]
Tareas-Trabajador asignado:   [0, 2, 0, 1]
Trabajadores-Tareas en orden: [[0, 2], [3], [1]]


In [10]:
## La primera mitad de las tareas las realiza el trabajador que decida el padre,
## la segunda mitad quien diga la madre

def cruza_padres(problema_genetico,padres):
    nueva_poblacion = []
    for i in range(0,len(padres),2):
        nueva_poblacion += problema_genetico.cruza(padres[i],padres[i+1])
    return nueva_poblacion

In [11]:
## Muta todos los individuos con una cierta probabilidad

def muta_individuos(problema_genetico, poblacion, prob):
    nueva_poblacion = []
    for individuo in poblacion:
        nueva_poblacion.append(problema_genetico.muta(individuo,prob))
    return nueva_poblacion

In [12]:
## Elige n elementos por torneo de k candidatos (grupos de torneo aleatorios)

def seleccion_por_torneo(problema_genetico, poblacion, n, k):
    seleccionados = []
    for i in range(n):
        participantes = random.sample(poblacion,k)
        seleccionado = min(participantes, key=problema_genetico.fitness)
        seleccionados.append(seleccionado)
    return seleccionados  

In [13]:
def algoritmo_genetico(problema_genetico,k,ngen,size,prop_cruces,prob_mutar):
    ## Generamos una población inicial
    poblacion= poblacion_inicial(problema_genetico,size)
    
    ## Tomamos el número de padres (siempre par) y no padres en base a la proporción dada
    n_padres=round(size*prop_cruces)
    n_padres= int(n_padres if n_padres%2==0 else n_padres-1)
    n_directos = size-n_padres
    
    ## Hacemos avanzar ngen generaciones nuestra población
    for _ in range(ngen):
        poblacion=nueva_generacion(problema_genetico,k,poblacion,n_padres, n_directos,prob_mutar)
    
    ## Nos quedamos con el mejor individuo en base a nuestra función de fitness
    ## devolvemos su fenotipo (decodificación) y valor (función fitness)
    mejor_cr= min(poblacion, key=problema_genetico.fitness)
    mejor=problema_genetico.decodifica(mejor_cr)
    return (mejor,problema_genetico.fitness(mejor_cr)) 


In [14]:
def nueva_generacion(problema_genetico, k, poblacion, n_padres, n_directos, prob_mutar):
    ## Realizamos la selección por torneo
    padres2 = seleccion_por_torneo(problema_genetico, poblacion, n_directos, k) 
    padres1 = seleccion_por_torneo(problema_genetico, poblacion, n_padres ,  k)
    ## Realizamos los cruces, la siguiente generación son los cruces de los padres y
    ## k elementos seleccionados por torneo de los no padres
    cruces =  cruza_padres(problema_genetico,padres1)
    generacion = padres2+cruces
    ## Aplicamos mutaciones
    resultado_mutaciones = muta_individuos(problema_genetico, generacion, prob_mutar)
    return resultado_mutaciones

In [15]:
grid = [[10,20,-1],[-1,-1,30]]
problemaAux = ProblemaGenetico(grid, [(0,1)])

In [16]:
#algoritmo_genetico(problema_genetico,k,ngen,size,prop_cruces,prob_mutar):
algoritmo_genetico(problemaAux,3,20,10,0.7,0.1)

AttributeError: 'NoneType' object has no attribute 'trab_tareasOrden'

In [17]:
cromo = Cromosoma([0,0,1],[[0,1],[2]])
print(cromo);

Tareas-Trabajador asignado:   [0, 0, 1]
Trabajadores-Tareas en orden: [[0, 1], [2]]
